In [ ]:
import numpy as np


In [ ]:
datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')

In [ ]:
print(datain)
print(datain.shape)   # Useful if it’s an array
print(type(datain)) 

In [ ]:
print(dataout)
print(dataout.shape)   # Useful if it’s an array
print(type(dataout)) 

# Week 1

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- Initial data ---
#X_init = np.array([...])  # your 15x3 input array
#y_init = np.array([...])  # your 15 outputs

X_init = datain
y_init = dataout

In [ ]:
# --- Fit Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_init, y_init)

# --- Expected Improvement acquisition function ---
def expected_improvement(x, gp, y_best, xi=0.01):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu = mu[0]
    sigma = sigma[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # negative because minimize()

# --- Optimize acquisition function with multiple random starts ---
y_best = y_init.max()
bounds = [(0,1), (0,1), (0,1)]

best_x = None
best_ei = float('inf')
for _ in range(10):
    x0 = np.random.rand(3)
    res = minimize(lambda x: expected_improvement(x, gp, y_best),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_ei:
        best_ei = res.fun
        best_x = res.x

x_next = best_x
print("Next compound combination to try:", x_next)


#Week 2

In [ ]:
import numpy as np

# Add the new observation
X_new = np.array([[0.901884, 0.273619, 0.723590]])
y_new = np.array([-0.13917950276182944])

# Combine with previous data
X_all = np.vstack([X_init, X_new])
y_all = np.concatenate([y_init, y_new])

X_init = X_all
y_init = y_all

print(X_init)
print(y_init)


In [ ]:
# --- Fit Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_init, y_init)

In [ ]:
# --- Expected Improvement acquisition function ---
def expected_improvement(x, gp, y_best, xi=0.01):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu = mu[0]
    sigma = sigma[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # negative because minimize()

In [ ]:
# --- Optimize acquisition function with multiple random starts ---
y_best = y_init.max()
bounds = [(0,1), (0,1), (0,1)]

best_x = None
best_ei = float('inf')
for _ in range(30):
    x0 = np.random.rand(3)
    res = minimize(lambda x: expected_improvement(x, gp, y_best, xi=0.01),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_ei:
        best_ei = res.fun
        best_x = res.x

x_next = best_x
print("Next compound combination to try:", x_next)

# Week 3

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.stats import norm
from scipy.optimize import minimize

# --- Initial data ---
X_init = np.array([[0.17152521, 0.34391687, 0.2487372 ],
                   [0.24211446, 0.64407427, 0.27243281],
                   [0.53490572, 0.39850092, 0.17338873],
                   [0.49258141, 0.61159319, 0.34017639],
                   [0.13462167, 0.21991724, 0.45820622],
                   [0.34552327, 0.94135983, 0.26936348],
                   [0.15183663, 0.43999062, 0.99088187],
                   [0.64550284, 0.39714294, 0.91977134],
                   [0.74691195, 0.28419631, 0.22629985],
                   [0.17047699, 0.6970324 , 0.14916943],
                   [0.22054934, 0.29782524, 0.34355534],
                   [0.66601366, 0.67198515, 0.2462953 ],
                   [0.04680895, 0.23136024, 0.77061759],
                   [0.60009728, 0.72513573, 0.06608864],
                   [0.96599485, 0.86111969, 0.56682913]])
y_init = np.array([-0.1121222 , -0.08796286, -0.11141465, -0.03483531, -0.04800758,
                   -0.11062091, -0.39892551, -0.11386851, -0.13146061, -0.09418956,
                   -0.04694741, -0.10596504, -0.11804826, -0.03637783, -0.05675837])

# --- Add last week's observation ---
X_init = np.vstack([X_init, [0.901884, 0.273619, 0.723590]])
y_init = np.concatenate([y_init, [-0.13917950276182944]])

# --- Add this week's observation ---
X_init = np.vstack([X_init, [0.393393, 0.209704, 0.929527]])
y_init = np.concatenate([y_init, [-0.1691006942213739]])

# --- Fit Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_init, y_init)

# --- Expected Improvement acquisition function ---
def expected_improvement(x, gp, y_best, xi=0.01):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu = mu[0]
    sigma = sigma[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # negative because minimize()

# --- Optimize acquisition function ---
bounds = [(0,1), (0,1), (0,1)]
y_best = y_init.max()

best_x = None
best_ei = float('inf')
for _ in range(50):
    x0 = np.random.rand(3)
    res = minimize(lambda x: expected_improvement(x, gp, y_best, xi=0.01),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_ei:
        best_ei = res.fun
        best_x = res.x

x_next = best_x
x_next_6dp = np.round(x_next, 6)
x_next_6dp

print("Next compound combination to try:", x_next)

In [ ]:
print("Next compound combination to try:", x_next_6dp)

# Week 4

New point: [0.117994, 0.128418, 0.478883]
Output: -0.04303671
This new point’s output (-0.043) is slightly worse than the best so far, but better than many other points
Exploration is yielding useful information: 
The GP now knows that very high or extreme values of compounds are likely to produce higher adverse reactions
so it can shift focus toward more moderate combinations.

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.stats import norm
from scipy.optimize import minimize

# --- Initial data ---
X_init = np.array([
    [0.17152521, 0.34391687, 0.2487372 ],
    [0.24211446, 0.64407427, 0.27243281],
    [0.53490572, 0.39850092, 0.17338873],
    [0.49258141, 0.61159319, 0.34017639],
    [0.13462167, 0.21991724, 0.45820622],
    [0.34552327, 0.94135983, 0.26936348],
    [0.15183663, 0.43999062, 0.99088187],
    [0.64550284, 0.39714294, 0.91977134],
    [0.74691195, 0.28419631, 0.22629985],
    [0.17047699, 0.6970324 , 0.14916943],
    [0.22054934, 0.29782524, 0.34355534],
    [0.66601366, 0.67198515, 0.2462953 ],
    [0.04680895, 0.23136024, 0.77061759],
    [0.60009728, 0.72513573, 0.06608864],
    [0.96599485, 0.86111969, 0.56682913]
])

y_init = np.array([
    -0.1121222 , -0.08796286, -0.11141465, -0.03483531, -0.04800758,
    -0.11062091, -0.39892551, -0.11386851, -0.13146061, -0.09418956,
    -0.04694741, -0.10596504, -0.11804826, -0.03637783, -0.05675837
])

# --- Add new observations from previous rounds ---
X_new = np.array([
    [0.901884, 0.273619, 0.723590],
    [0.393393, 0.209704, 0.929527],
    [0.117994, 0.128418, 0.478883],
    [0.442975, 0.132914, 0.276531]
])

y_new = np.array([
    -0.13917950276182944,
    -0.1691006942213739,
    -0.043036710546582216,
    -0.11152705861024295
])

# Combine all data
X_all = np.vstack([X_init, X_new])
y_all = np.concatenate([y_init, y_new])

# --- Fit Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_all, y_all)

# --- Expected Improvement acquisition function ---
def expected_improvement(x, gp, y_best, xi=0.01):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu, sigma = mu[0], sigma[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # minimize

# --- Optimize acquisition function ---
bounds = [(0, 1), (0, 1), (0, 1)]
y_best = y_all.max()

best_x, best_ei = None, float('inf')
for _ in range(50):  # multiple random restarts
    x0 = np.random.rand(3)
    res = minimize(lambda x: expected_improvement(x, gp, y_best, xi=0.01),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_ei:
        best_ei, best_x = res.fun, res.x

x_next = np.round(best_x, 6)
print("Next compound combination to try:", x_next)


# Week 5

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.stats import norm
from scipy.optimize import minimize

# --- Initial data ---
X_init = np.array([
    [0.17152521, 0.34391687, 0.2487372 ],
    [0.24211446, 0.64407427, 0.27243281],
    [0.53490572, 0.39850092, 0.17338873],
    [0.49258141, 0.61159319, 0.34017639],
    [0.13462167, 0.21991724, 0.45820622],
    [0.34552327, 0.94135983, 0.26936348],
    [0.15183663, 0.43999062, 0.99088187],
    [0.64550284, 0.39714294, 0.91977134],
    [0.74691195, 0.28419631, 0.22629985],
    [0.17047699, 0.6970324 , 0.14916943],
    [0.22054934, 0.29782524, 0.34355534],
    [0.66601366, 0.67198515, 0.2462953 ],
    [0.04680895, 0.23136024, 0.77061759],
    [0.60009728, 0.72513573, 0.06608864],
    [0.96599485, 0.86111969, 0.56682913]
])

y_init = np.array([
    -0.1121222 , -0.08796286, -0.11141465, -0.03483531, -0.04800758,
    -0.11062091, -0.39892551, -0.11386851, -0.13146061, -0.09418956,
    -0.04694741, -0.10596504, -0.11804826, -0.03637783, -0.05675837
])

# --- Add new observations from previous rounds ---
X_new = np.array([
    [0.901884, 0.273619, 0.723590],
    [0.393393, 0.209704, 0.929527],
    [0.117994, 0.128418, 0.478883],
    [0.442975, 0.132914, 0.276531]
])

y_new = np.array([
    -0.13917950276182944,
    -0.1691006942213739,
    -0.043036710546582216,
    -0.11152705861024295
])

# Combine all data
X_all = np.vstack([X_init, X_new])
y_all = np.concatenate([y_init, y_new])

- Not an improvement: the best observed transformed value so far is about −0.0348
- (a less negative value is better because we’re maximizing -side_effects),
- so this new point is worse than the current best.
- The GP posterior will reduce uncertainty in the neighborhood
- Practically this pushes the model to focus exploration elsewhere: (a) neighborhoods around the current best (~0.49,0.61,0.34 and the new good point ~0.118,0.128,0.479), or (b) regions that remain uncertain (e.g. combinations with small/large first compound and moderate others).
- Go with more exploitation (fine-tuning around the best), keep xi small (0.01).

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.stats import norm
from scipy.optimize import minimize

# X_all, y_all should be your current full dataset (including the latest point)
# Example: start from your known arrays and append the new point
# X_all = np.vstack([previous_X, [0.442975, 0.132914, 0.276531]])
# y_all = np.concatenate([previous_y, [-0.11152705861024295]])

# --- Fit GP ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_all, y_all)

# --- Expected Improvement ---
def expected_improvement(x, gp, y_best, xi=0.01):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu, sigma = mu[0], sigma[0]
    if sigma <= 1e-12:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # minimize

# --- Optimize EI ---
bounds = [(0,1),(0,1),(0,1)]
y_best = y_all.max()  # since we maximize -side_effects

best_x, best_ei = None, float('inf')
for _ in range(50):
    x0 = np.random.rand(3)
    res = minimize(lambda x: expected_improvement(x, gp, y_best, xi=0.01),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.success and res.fun < best_ei:
        best_ei = res.fun
        best_x = res.x

print("Next candidate (6 dp):", np.round(best_x, 6))


# Week 6
- New input: [0.683163, 0.340182, 0.318758]
- New output: -0.05490061720239002
- Current best observed (so far): -0.03483531 (from [0.49258141, 0.61159319, 0.34017639])
- Worst observed: ~-0.3989 (from [0.15183663, 0.43999062, 0.99088187])
- So the new point is not an improvement over the best (it’s ~0.02007 worse), but it’s far from the worst — a mid-to-better-range sample.

The new point lies in a region with relatively large first compound, moderate second, and moderate third.
Its output (~−0.055) is consistent with the pattern we’ve been seeing:
Best values tend to appear for moderate compound mixes (e.g. the best ~−0.034 at ~[0.49,0.61,0.34] 
                                                        and the recent decent point ~−0.043 at ~[0.118,0.128,0.479]).
Extremely high third-compound values tended to be bad (e.g. the −0.399 sample with a very high third compound).

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.stats import norm
from scipy.optimize import minimize

# --- Initial data ---
X_init = np.array([
    [0.17152521, 0.34391687, 0.2487372 ],
    [0.24211446, 0.64407427, 0.27243281],
    [0.53490572, 0.39850092, 0.17338873],
    [0.49258141, 0.61159319, 0.34017639],
    [0.13462167, 0.21991724, 0.45820622],
    [0.34552327, 0.94135983, 0.26936348],
    [0.15183663, 0.43999062, 0.99088187],
    [0.64550284, 0.39714294, 0.91977134],
    [0.74691195, 0.28419631, 0.22629985],
    [0.17047699, 0.6970324 , 0.14916943],
    [0.22054934, 0.29782524, 0.34355534],
    [0.66601366, 0.67198515, 0.2462953 ],
    [0.04680895, 0.23136024, 0.77061759],
    [0.60009728, 0.72513573, 0.06608864],
    [0.96599485, 0.86111969, 0.56682913]

    [1.       0.934185 0.44019 ]
])

y_init = np.array([
    -0.1121222 , -0.08796286, -0.11141465, -0.03483531, -0.04800758,
    -0.11062091, -0.39892551, -0.11386851, -0.13146061, -0.09418956,
    -0.04694741, -0.10596504, -0.11804826, -0.03637783, -0.05675837
])

# --- Add new observations from previous rounds ---
X_new = np.array([
    [0.901884, 0.273619, 0.723590],
    [0.393393, 0.209704, 0.929527],
    [0.117994, 0.128418, 0.478883],
    [0.442975, 0.132914, 0.276531],
    [0.683163, 0.340182, 0.318758]
])

y_new = np.array([
    -0.13917950276182944,
    -0.1691006942213739,
    -0.043036710546582216,
    -0.11152705861024295,
    -0.05490061720239002
])

# Combine all data
X_all = np.vstack([X_init, X_new])
y_all = np.concatenate([y_init, y_new])

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.stats import norm
from scipy.optimize import minimize

# X_all, y_all should be your current full dataset (including the latest point)
# Example: start from your known arrays and append the new point
# X_all = np.vstack([previous_X, [0.442975, 0.132914, 0.276531]])
# y_all = np.concatenate([previous_y, [-0.11152705861024295]])

# --- Fit GP ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_all, y_all)

# --- Expected Improvement ---
def expected_improvement(x, gp, y_best, xi=0.01):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu, sigma = mu[0], sigma[0]
    if sigma <= 1e-12:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # minimize

# --- Optimize EI ---
bounds = [(0,1),(0,1),(0,1)]
y_best = y_all.max()  # since we maximize -side_effects

best_x, best_ei = None, float('inf')
for _ in range(50):
    x0 = np.random.rand(3)
    res = minimize(lambda x: expected_improvement(x, gp, y_best, xi=0.01),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.success and res.fun < best_ei:
        best_ei = res.fun
        best_x = res.x

print("Next candidate (6 dp):", np.round(best_x, 6))


- inspect μ, σ, EI at [0.974184 0.825897 0.656309] and rank a bunch of candidate points so you can see if the selected point truly had the highest EI
- or if optimizer artifacts were involved:

In [ ]:
import numpy as np
from scipy.stats import norm

# gp = your trained GaussianProcessRegressor
# y_best = your current best (y.max())
# xi = 0.01

x_candidate = np.array([0.974184, 0.825897, 0.656309]).reshape(1, -1)

mu, sigma = gp.predict(x_candidate, return_std=True)
mu = float(mu[0])
sigma = float(sigma[0])

def EI_from_mu_sigma(mu, sigma, y_best, xi=0.01):
    if sigma <= 1e-12:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    return imp * norm.cdf(Z) + sigma * norm.pdf(Z)

ei_value = EI_from_mu_sigma(mu, sigma, y_best, xi=0.01)

print("Candidate x:", x_candidate.ravel())
print("Predicted mean (mu):", mu)
print("Predicted std  (sigma):", sigma)
print("EI at candidate:", ei_value)

# OPTIONAL: test a grid of random candidates and show top 10 by EI
cands = np.random.rand(2000, 3)
mus, sigs = gp.predict(cands, return_std=True)
eis = [EI_from_mu_sigma(m, s, y_best, xi=0.01) for m, s in zip(mus, sigs)]
top_idx = np.argsort(eis)[-10:][::-1]
print("\nTop 10 candidates found (EI, mu, sigma, x):")
for i in top_idx:
    print(f"{eis[i]:.6e}, {mus[i]:.6e}, {sigs[i]:.6e}, {np.round(cands[i],6)}")


In [ ]:
print(gp.kernel_)

# Rewire summary

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, Matern
from scipy.stats import norm
from scipy.optimize import minimize

# --------------------------------------------------
# 1. Load / continue your dataset
# --------------------------------------------------
X = X_all   # your current N x 3 inputs
y = y_all   # your current N outputs

# --------------------------------------------------
# 2. Refit GP with bounded length-scales
# --------------------------------------------------
kernel = ConstantKernel(1.0, (1e-3, 1e3)) * Matern(
    length_scale=[0.3, 0.3, 0.3],          # starting guess
    length_scale_bounds=(1e-2, 1.0),       # prevents collapse
    nu=2.5
)

gp = GaussianProcessRegressor(
    kernel=kernel,
    alpha=1e-4,             # slightly more noise to avoid overfitting
    normalize_y=True,
    n_restarts_optimizer=15 # stronger hyperparameter search
)

gp.fit(X, y)

print("Fitted kernel:", gp.kernel_)

# --------------------------------------------------
# 3. Expected Improvement
# --------------------------------------------------
def expected_improvement(x, gp, y_best, xi=0.05):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu, sigma = mu[0], sigma[0]

    if sigma < 1e-12:
        return 0.0

    imp = mu - y_best - xi
    Z = imp / sigma
    return imp * norm.cdf(Z) + sigma * norm.pdf(Z)

y_best = y.max()

# --------------------------------------------------
# 4. Global search: random Sobol-like candidate set
# --------------------------------------------------
num_candidates = 3000
candidates = np.random.rand(num_candidates, 3)

EI_values = np.array([expected_improvement(x, gp, y_best) for x in candidates])

# Pick top K seeds for refinement
K = 10
top_idx = np.argsort(EI_values)[-K:]
seeds = candidates[top_idx]

# --------------------------------------------------
# 5. Local optimization from top seeds
# --------------------------------------------------
bounds = [(0,1), (0,1), (0,1)]
best_x = None
best_ei = -1e9

for seed in seeds:
    res = minimize(lambda x: -expected_improvement(x, gp, y_best),
                   x0=seed, bounds=bounds, method="L-BFGS-B")
    ei = -res.fun
    if ei > best_ei:
        best_ei = ei
        best_x = res.x

# --------------------------------------------------
# 6. Print the final next point
# --------------------------------------------------
x_next = np.round(best_x, 6)
print("\nNext compound combination to test:", x_next)
print("Expected Improvement:", best_ei)


But your results also reveal a clear model problem (the tiny length-scale)
that makes me cautious about trusting that extreme recommendation.

In [ ]:
x = np.array([[1.0, 0.934734, 0.440159]])
mu, sigma = gp.predict(x, return_std=True)
print("mu =", mu[0], " sigma =", sigma[0])
# EI calculation
from scipy.stats import norm
y_best = y_all.max()
xi = 0.01
imp = mu[0] - y_best - xi
Z = imp / sigma[0] if sigma[0] > 1e-12 else 0.0
ei = imp * norm.cdf(Z) + sigma[0] * norm.pdf(Z) if sigma[0] > 1e-12 else 0.0
print("EI =", ei)


Use a large quasi-random sample (Sobol or random) and rank EI to ensure the optimizer didn’t miss a better basin:

In [ ]:
import numpy as np
cands = np.random.rand(20000, 3)   # or use Sobol sequence if available
mus, sigs = gp.predict(cands, return_std=True)
def EI_from_mu_sigma(mu, sigma, y_best, xi=0.01):
    if sigma <= 1e-12:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    return imp * norm.cdf(Z) + sigma * norm.pdf(Z)
eis = np.array([EI_from_mu_sigma(m,s,y_best) for m,s in zip(mus,sigs)])
top_idx = np.argsort(eis)[-20:][::-1]
for i in top_idx[:10]:
    print(eis[i], mus[i], sigs[i], np.round(cands[i],6))


I can give you the exact code that (1) refits the GP with bounds, (2) computes top-EI candidates via Sobol + local refinement, and (3) returns a robust next choice (top-k + μ ranking). Want me to produce that full snippet now?

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, Matern, WhiteKernel
from scipy.stats import norm
from scipy.optimize import minimize

# X_all and y_all should be defined already
X = X_all
y = y_all

# --------------------------------------------------
# Fit GP with safe length-scale bounds
# --------------------------------------------------
kernel = ConstantKernel(1.0, (1e-3, 1e3)) * Matern(
    length_scale=[0.3, 0.3, 0.3],         
    length_scale_bounds=(1e-2, 1.0),
    nu=2.5
) + WhiteKernel(noise_level=1e-6, noise_level_bounds=(1e-8, 1e-2))

gp = GaussianProcessRegressor(
    kernel=kernel,
    normalize_y=True,
    n_restarts_optimizer=12
)

gp.fit(X, y)
print("Fitted kernel:", gp.kernel_)

# --------------------------------------------------
# Expected Improvement
# --------------------------------------------------
def EI(x, gp, y_best, xi=0.05):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu, sigma = mu[0], sigma[0]
    if sigma < 1e-12:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    return imp * norm.cdf(Z) + sigma * norm.pdf(Z)

y_best = y.max()

# --------------------------------------------------
# Global candidate sampling
# --------------------------------------------------
N_global = 5000
global_candidates = np.random.rand(N_global, 3)

mus, sigmas = gp.predict(global_candidates, return_std=True)

# Compute EI for all candidates
ei_values = np.array([
    EI(x, gp, y_best) for x in global_candidates
])

# Select best EI candidates as seeds
K = 20
top_idx = np.argsort(ei_values)[-K:]
seeds = global_candidates[top_idx]

# --------------------------------------------------
# Local refinement (L-BFGS)
# --------------------------------------------------
bounds = [(0,1), (0,1), (0,1)]
best_x = None
best_ei = -1e9

for seed in seeds:
    res = minimize(lambda x: -EI(x, gp, y_best),
                   x0=seed,
                   bounds=bounds,
                   method="L-BFGS-B")
    if res.success:
        val = -res.fun
        if val > best_ei:
            best_ei = val
            best_x = res.x

x_next = np.round(best_x, 6)

print("\nNext x to test:", x_next)
print("EI at x_next:", best_ei)


- That point is no good
- re-fit the GP with reasonable length-scale bounds (or a different kernel/noise model) and re-run EI maximization with more robust search. That will produce far more meaningful next candidates.

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.stats import norm
from scipy.optimize import minimize

# ---------------------------
# 1. Define Expected Improvement
# ---------------------------
def expected_improvement(x, gp, y_best, xi=0.01):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu, sigma = mu[0], sigma[0]
    if sigma < 1e-12:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # negative for minimization


# ---------------------------
# 2. Fit GP with bounded kernel to prevent collapse
# ---------------------------
kernel = Matern(length_scale=0.2, length_scale_bounds=(1e-2, 1e2), nu=2.5)
gp = GaussianProcessRegressor(
    kernel=kernel,
    alpha=1e-6,
    normalize_y=True,
    n_restarts_optimizer=15
)
gp.fit(X_all, y_all)

print("Fitted kernel:", gp.kernel_)

y_best = y_all.max()


# ---------------------------
# 3. Global EI Search via Random Starts + Local Refinement
# ---------------------------
best_ei = -1e9
best_x = None

# Sobol-like sampling
n_candidates = 3000
candidates = np.random.rand(n_candidates, 3)

for x0 in candidates:
    res = minimize(expected_improvement, x0,
                   args=(gp, y_best),
                   bounds=[(0,1),(0,1),(0,1)],
                   method="L-BFGS-B",
                   options={'maxiter': 50})
    ei_val = -res.fun
    if ei_val > best_ei:
        best_ei = ei_val
        best_x = res.x

# Round for reporting
x_next = np.round(best_x, 6)

mu_next, sigma_next = gp.predict(x_next.reshape(1,-1), return_std=True)

print("\nNext point to evaluate:", x_next)
print("Predicted mean (mu):", float(mu_next))
print("Predicted std (sigma):", float(sigma_next))
print("EI:", float(best_ei))


- x_next = [0.514909, 0.429977, 0.441068] is a reasonable, expected selection
- the GP predicts slightly better performance there and retains enough uncertainty that the expected improvement is positive and larger than earlier extreme choices. 

# Week 7

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.stats import norm
from scipy.optimize import minimize

# --- Initial data ---
X_init = np.array([
    [0.17152521, 0.34391687, 0.2487372 ],
    [0.24211446, 0.64407427, 0.27243281],
    [0.53490572, 0.39850092, 0.17338873],
    [0.49258141, 0.61159319, 0.34017639],
    [0.13462167, 0.21991724, 0.45820622],
    [0.34552327, 0.94135983, 0.26936348],
    [0.15183663, 0.43999062, 0.99088187],
    [0.64550284, 0.39714294, 0.91977134],
    [0.74691195, 0.28419631, 0.22629985],
    [0.17047699, 0.6970324 , 0.14916943],
    [0.22054934, 0.29782524, 0.34355534],
    [0.66601366, 0.67198515, 0.2462953 ],
    [0.04680895, 0.23136024, 0.77061759],
    [0.60009728, 0.72513573, 0.06608864],
    [0.96599485, 0.86111969, 0.56682913]
])

y_init = np.array([
    -0.1121222 , -0.08796286, -0.11141465, -0.03483531, -0.04800758,
    -0.11062091, -0.39892551, -0.11386851, -0.13146061, -0.09418956,
    -0.04694741, -0.10596504, -0.11804826, -0.03637783, -0.05675837
])

# --- Add new observations from previous rounds ---
X_new = np.array([
    [0.901884, 0.273619, 0.723590],
    [0.393393, 0.209704, 0.929527],
    [0.117994, 0.128418, 0.478883],
    [0.442975, 0.132914, 0.276531],
    [0.683163, 0.340182, 0.318758],
    [0.514909, 0.429977, 0.441068]
])

y_new = np.array([
    -0.13917950276182944,
    -0.1691006942213739,
    -0.043036710546582216,
    -0.11152705861024295,
    -0.05490061720239002, 
    -0.0050467853497373665
])

# Combine all data
X_all = np.vstack([X_init, X_new])
y_all = np.concatenate([y_init, y_new])

- this is a new best.
- -0.0050468 − (−0.03483531) = +0.0297885 — a substantial improvement in the transformed objective.
- You just discovered a substantially better compound mix — much better than the GP expected — so retrain, replicate, then focus sampling locally around [0.514909, 0.429977, 0.441068] while keeping a small exploration budget elsewhere.

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.stats import norm
from scipy.optimize import minimize

# Suppose X_all, y_all are your current arrays (include the new point)

# Refit GP
kernel = Matern(length_scale=0.205, nu=2.5)  # or kernel=Matern(nu=2.5) to allow fitting
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_all, y_all)

# Update best
y_best = y_all.max()

# EI
def expected_improvement(x, gp, y_best, xi=0.01):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu, sigma = mu[0], sigma[0]
    if sigma <= 1e-12:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    return imp * norm.cdf(Z) + sigma * norm.pdf(Z)

# Optimize -EI (use many restarts; also try seeding with Sobol/random grid)
bounds = [(0,1),(0,1),(0,1)]
best_x, best_ei = None, -np.inf
for _ in range(100):
    x0 = np.random.rand(3)
    res = minimize(lambda x: -expected_improvement(x, gp, y_best, xi=0.01),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.success:
        ei_val = -res.fun
        if ei_val > best_ei:
            best_ei = ei_val
            best_x = res.x

print("Next x (6 dp):", np.round(best_x, 6))
print("EI at x_next:", best_ei)


# Week 8

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.stats import norm
from scipy.optimize import minimize

# --- Initial data ---
X_init = np.array([
    [0.17152521, 0.34391687, 0.2487372 ],
    [0.24211446, 0.64407427, 0.27243281],
    [0.53490572, 0.39850092, 0.17338873],
    [0.49258141, 0.61159319, 0.34017639],
    [0.13462167, 0.21991724, 0.45820622],
    [0.34552327, 0.94135983, 0.26936348],
    [0.15183663, 0.43999062, 0.99088187],
    [0.64550284, 0.39714294, 0.91977134],
    [0.74691195, 0.28419631, 0.22629985],
    [0.17047699, 0.6970324 , 0.14916943],
    [0.22054934, 0.29782524, 0.34355534],
    [0.66601366, 0.67198515, 0.2462953 ],
    [0.04680895, 0.23136024, 0.77061759],
    [0.60009728, 0.72513573, 0.06608864],
    [0.96599485, 0.86111969, 0.56682913]
])

y_init = np.array([
    -0.1121222 , -0.08796286, -0.11141465, -0.03483531, -0.04800758,
    -0.11062091, -0.39892551, -0.11386851, -0.13146061, -0.09418956,
    -0.04694741, -0.10596504, -0.11804826, -0.03637783, -0.05675837
])

# --- Add new observations from previous rounds ---
X_new = np.array([
    [0.901884, 0.273619, 0.723590],
    [0.393393, 0.209704, 0.929527],
    [0.117994, 0.128418, 0.478883],
    [0.442975, 0.132914, 0.276531],
    [0.683163, 0.340182, 0.318758],
    [0.514909, 0.429977, 0.441068],
    [0.376435, 0.333454, 0.472888]
])

y_new = np.array([
    -0.13917950276182944,
    -0.1691006942213739,
    -0.043036710546582216,
    -0.11152705861024295,
    -0.05490061720239002, 
    -0.0050467853497373665,
    -0.017389844227996235
])

# Combine all data
X_all = np.vstack([X_init, X_new])
y_all = np.concatenate([y_init, y_new])

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.stats import norm
from scipy.optimize import minimize

# --- Fit GP (allow automatic length-scale optimization) ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_all, y_all)

# --- Best observed value ---
y_best = y_all.max()

# --- Expected Improvement ---
def expected_improvement(x, gp, y_best, xi=0.01):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu, sigma = mu[0], sigma[0]

    if sigma < 1e-12:
        return 0.0

    imp = mu - y_best - xi
    if imp <= 0:
        return 0.0   # no improvement possible
    
    Z = imp / sigma
    return imp * norm.cdf(Z) + sigma * norm.pdf(Z)

# --- Optimize EI ---
bounds = [(0,1),(0,1),(0,1)]
best_x, best_ei = None, -np.inf

for _ in range(100):
    x0 = np.random.rand(3)
    res = minimize(lambda x: -expected_improvement(x, gp, y_best),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.success:
        ei_val = -res.fun
        if ei_val > best_ei:
            best_ei = ei_val
            best_x = res.x

print("Next x (6 dp):", np.round(best_x, 6))
print("EI at x_next:", best_ei)
print("Fitted kernel:", gp.kernel_)


In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.stats import norm
from scipy.optimize import minimize

# make runs reproducible
np.random.seed(0)
RND = np.random.RandomState(0)

# sensible kernel with bounds for length_scale to avoid collapse
kernel = Matern(length_scale=0.2, length_scale_bounds=(1e-2, 2.0), nu=2.5)

# add small jitter/noise term so predictive sigma isn't exactly zero
gp = GaussianProcessRegressor(
    kernel=kernel,
    alpha=1e-4,                 # larger than 1e-6 to regularize
    normalize_y=True,
    n_restarts_optimizer=10,    # try multiple restarts for kernel fitting
    random_state=0
)

gp.fit(X_all, y_all)

# recompute y_best
y_best = y_all.max()

def ei_at(x, gp, y_best, xi=0.01):
    x = np.array(x).reshape(1,-1)
    mu, sigma = gp.predict(x, return_std=True)
    mu, sigma = float(mu[0]), float(sigma[0])
    if sigma <= 1e-12:
        return 0.0, mu, sigma
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return float(ei), mu, sigma

# deterministic candidate seeding (Sobol or grid would be better; here: reproducible random seeds)
best_x, best_ei = None, -np.inf
for _ in range(200):               # more seeds for robustness
    x0 = RND.rand(3)
    res = minimize(lambda x: -ei_at(x, gp, y_best, xi=0.01)[0],
                   x0=x0, bounds=[(0,1)]*3, method='L-BFGS-B',
                   options={'maxiter':200})
    if res.success:
        ei_val, mu_val, sig_val = ei_at(res.x, gp, y_best, xi=0.01)
        if ei_val > best_ei:
            best_ei = ei_val
            best_x = res.x
            best_mu, best_sigma = mu_val, sig_val

print("Next x (6 dp):", np.round(best_x, 6))
print("EI at x_next:", best_ei)
print("mu:", best_mu, "sigma:", best_sigma)
print("fitted kernel:", gp.kernel_)


- The above code is not working.
- EI = 0 everywhere	GP overfitting → σ→0
- Kernel fitted as length_scale=1e-05 led to hyperparameter collapse
- Had a reset and used the below
- [0.514909, 0.429977, 0.441068] is the current best point, so points in its vicinity are highly exploitable.
- [0.376435, 0.333454, 0.472888] is slightly worse but still promising — can also be exploited with small perturbations.
- [0.117994, 0.128418, 0.478883] and [0.49258141, 0.61159319, 0.34017639] are secondary exploitation candidates.

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.stats import norm
from scipy.optimize import minimize

# --------------------------
# Suppose X_all, y_all already contain all previous points
# --------------------------

# Set random seed for reproducibility
RND = np.random.RandomState(42)

# --- Fit GP with Matern kernel ---
kernel = Matern(length_scale=0.2, length_scale_bounds=(1e-2, 2.0), nu=2.5)
gp = GaussianProcessRegressor(
    kernel=kernel,
    alpha=1e-4,              # small regularization
    normalize_y=True,
    n_restarts_optimizer=10,
    random_state=RND
)
gp.fit(X_all, y_all)

# --- Current best ---
y_best = y_all.max()

# --- EI function (leaning toward exploitation by small xi) ---
def ei_exploit(x, gp, y_best, xi=0.001):  # smaller xi => more exploitation
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu, sigma = float(mu[0]), float(sigma[0])
    if sigma <= 1e-12:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return ei

# --- Optimize EI ---
best_x, best_ei = None, -np.inf

# --- Seeding near current best points for exploitation ---
seed_points = [
    X_all[np.argmax(y_all)],             # best point
    X_all[np.argsort(y_all)[-2]],        # second-best
    X_all[np.argsort(y_all)[-3]]         # third-best
]

for seed in seed_points + [RND.rand(3) for _ in range(10)]:  # small number of random seeds
    res = minimize(lambda x: -ei_exploit(x, gp, y_best, xi=0.001),
                   x0=seed,
                   bounds=[(0,1)]*3,
                   method='L-BFGS-B')
    if res.success:
        ei_val = ei_exploit(res.x, gp, y_best, xi=0.001)
        if ei_val > best_ei:
            best_ei = ei_val
            best_x = res.x

print("Next point (6 dp):", np.round(best_x, 6))
print("EI at next point:", best_ei)


# Week 9
- 

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.stats import norm
from scipy.optimize import minimize

# --- Initial data ---
X_init = np.array([
    [0.17152521, 0.34391687, 0.2487372 ],
    [0.24211446, 0.64407427, 0.27243281],
    [0.53490572, 0.39850092, 0.17338873],
    [0.49258141, 0.61159319, 0.34017639],
    [0.13462167, 0.21991724, 0.45820622],
    [0.34552327, 0.94135983, 0.26936348],
    [0.15183663, 0.43999062, 0.99088187],
    [0.64550284, 0.39714294, 0.91977134],
    [0.74691195, 0.28419631, 0.22629985],
    [0.17047699, 0.6970324 , 0.14916943],
    [0.22054934, 0.29782524, 0.34355534],
    [0.66601366, 0.67198515, 0.2462953 ],
    [0.04680895, 0.23136024, 0.77061759],
    [0.60009728, 0.72513573, 0.06608864],
    [0.96599485, 0.86111969, 0.56682913]
])

y_init = np.array([
    -0.1121222 , -0.08796286, -0.11141465, -0.03483531, -0.04800758,
    -0.11062091, -0.39892551, -0.11386851, -0.13146061, -0.09418956,
    -0.04694741, -0.10596504, -0.11804826, -0.03637783, -0.05675837
])

# --- Add new observations from previous rounds ---
X_new = np.array([
    [0.901884, 0.273619, 0.723590],
    [0.393393, 0.209704, 0.929527],
    [0.117994, 0.128418, 0.478883],
    [0.442975, 0.132914, 0.276531],
    [0.683163, 0.340182, 0.318758],
    [0.514909, 0.429977, 0.441068],
    [0.376435, 0.333454, 0.472888],
    [0.597924, 0.348150, 0.493042]
])

y_new = np.array([
    -0.13917950276182944,
    -0.1691006942213739,
    -0.043036710546582216,
    -0.11152705861024295,
    -0.05490061720239002, 
    -0.0050467853497373665,
    -0.017389844227996235,
    -0.009890240573922564
])

# Combine all data
X_all = np.vstack([X_init, X_new])
y_all = np.concatenate([y_init, y_new])


- New point: [0.597924, 0.348150, 0.493042] Output: -0.009890240573922564
- This is one of the best points so far (second only to [0.514909,0.429977,0.441068] which gave -0.0050468).
- It strengthens the hypothesis that a moderate mix (x1 ≈ 0.4–0.6, x2 ≈ 0.3–0.45, x3 ≈ 0.4–0.5) is a promising region.

# Tweaks
- Exploit locally: run an exploit-leaning EI (xi small, seeded near best points). That will fine-tune the local optimum.
- Use a small perturbation strategy: sample a few small-step neighbors around the two top points to see if you can improve further.

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.stats import norm

# assume X_all, y_all already include the new point
# define candidates (as shown above)
candidates = np.array([
    [0.544909, 0.429977, 0.441068],
    [0.484909, 0.399977, 0.441068],
    [0.514909, 0.429977, 0.471068],
    [0.627924, 0.348150, 0.493042],
    [0.597924, 0.318150, 0.493042],
    [0.597924, 0.348150, 0.463042]
])

# fit GP (use the kernel settings you prefer)
kernel = Matern(length_scale=0.2, length_scale_bounds=(1e-2, 2.0), nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-4, normalize_y=True, n_restarts_optimizer=8, random_state=0)
gp.fit(X_all, y_all)

y_best = y_all.max()
xi = 0.001  # small -> exploit

def ei(mu, sigma, y_best, xi=0.001):
    if sigma <= 1e-12:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    return imp * norm.cdf(Z) + sigma * norm.pdf(Z)

mus, sigmas = gp.predict(candidates, return_std=True)
eis = [ei(mu, s, y_best, xi) for mu, s in zip(mus, sigmas)]

for x, mu, s, e in zip(candidates, mus, sigmas, eis):
    print("x:", np.round(x,6), " mu:", mu, " sigma:", s, " EI:", e)


| x                              | EI           | What it means                                       |
| ------------------------------ | ------------ | --------------------------------------------------- |
| [0.484909, 0.399977, 0.441068] | **0.002692** | Highest EI — best tradeoff of low mu & decent sigma |
| [0.544909, 0.429977, 0.441068] | 0.002538     | Also strong EI → close to best region               |
| [0.514909, 0.429977, 0.471068] | 0.002567     | Very similar region                                 |
| [0.597924, 0.34815, 0.463042]  | 0.001626     | Moderate EI — somewhat promising                    |
| [0.627924, 0.34815, 0.493042]  | 0.000543     | Weak EI — predicted worse                           |
| [0.597924, 0.31815, 0.493042]  | 0.000606     | Weak EI — predicted worse                           |


# Week 10
| Point                              | Observed y          |
| ---------------------------------- | ------------------- |
| [0.514909, 0.429977, 0.441068]     | **−0.00505** ← best |
| [0.597924, 0.348150, 0.493042]     | −0.00989            |
| [0.376435, 0.333454, 0.472888]     | −0.01739            |
| **[0.484909, 0.399977, 0.441068]** | **−0.02634**        |

This pattern suggests:
- There is a narrow local optimum near the best point
- Small moves away degrade performance quickly
- The surface is steep, not flat
- I want to tighten the kernel / length-scale to better reflect the steepness you’re seeing.

This means:
- The function is not very smooth locally.
- The GP should not assume long-range smoothness.
- Your model should expect short length-scales.
This explains earlier issues like:
- EI drifting to boundaries
- Flat EI surfaces
- σ collapsing or becoming uniform
- Optimizer instability
All of these are classic symptoms of a length-scale mismatch.

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.stats import norm
from scipy.optimize import minimize

# --- Initial data ---
X_init = np.array([
    [0.17152521, 0.34391687, 0.2487372 ],
    [0.24211446, 0.64407427, 0.27243281],
    [0.53490572, 0.39850092, 0.17338873],
    [0.49258141, 0.61159319, 0.34017639],
    [0.13462167, 0.21991724, 0.45820622],
    [0.34552327, 0.94135983, 0.26936348],
    [0.15183663, 0.43999062, 0.99088187],
    [0.64550284, 0.39714294, 0.91977134],
    [0.74691195, 0.28419631, 0.22629985],
    [0.17047699, 0.6970324 , 0.14916943],
    [0.22054934, 0.29782524, 0.34355534],
    [0.66601366, 0.67198515, 0.2462953 ],
    [0.04680895, 0.23136024, 0.77061759],
    [0.60009728, 0.72513573, 0.06608864],
    [0.96599485, 0.86111969, 0.56682913]
])

y_init = np.array([
    -0.1121222 , -0.08796286, -0.11141465, -0.03483531, -0.04800758,
    -0.11062091, -0.39892551, -0.11386851, -0.13146061, -0.09418956,
    -0.04694741, -0.10596504, -0.11804826, -0.03637783, -0.05675837
])

# --- Add new observations from previous rounds ---
X_new = np.array([
    [0.901884, 0.273619, 0.723590],
    [0.393393, 0.209704, 0.929527],
    [0.117994, 0.128418, 0.478883],
    [0.442975, 0.132914, 0.276531],
    [0.683163, 0.340182, 0.318758],
    [0.514909, 0.429977, 0.441068],
    [0.376435, 0.333454, 0.472888],
    [0.597924, 0.348150, 0.493042],
    [0.484909, 0.399977, 0.441068]
])

y_new = np.array([
    -0.13917950276182944,
    -0.1691006942213739,
    -0.043036710546582216,
    -0.11152705861024295,
    -0.05490061720239002, 
    -0.0050467853497373665,
    -0.017389844227996235,
    -0.009890240573922564,
    -0.02634297118984294
])

# Combine all data
X_all = np.vstack([X_init, X_new])
y_all = np.concatenate([y_init, y_new])


In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.stats import norm
from scipy.optimize import minimize

# -------------------------
# Reproducibility
# -------------------------
np.random.seed(0)
RND = np.random.RandomState(0)

# -------------------------
# Tightened kernel
# - shorter length_scale to model steep local variation
# - bounded to prevent collapse
# -------------------------
kernel = Matern(
    length_scale=0.15,
    length_scale_bounds=(0.05, 0.5),
    nu=2.5
)

gp = GaussianProcessRegressor(
    kernel=kernel,
    alpha=1e-4,                 # regularization to avoid sigma collapse
    normalize_y=True,
    n_restarts_optimizer=10,
    random_state=0
)

gp.fit(X_all, y_all)

print("Fitted kernel:", gp.kernel_)

# -------------------------
# Best observed value
# -------------------------
y_best = y_all.max()

# -------------------------
# Exploitation-leaning EI
# - very small xi
# -------------------------
def expected_improvement(x, gp, y_best, xi=0.001):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu, sigma = float(mu[0]), float(sigma[0])

    if sigma <= 1e-12:
        return 0.0

    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return ei

# -------------------------
# Optimization
# - seed heavily near best points
# -------------------------
bounds = [(0, 1), (0, 1), (0, 1)]

best_x, best_ei = None, -np.inf

# top 3 observed points (exploitation anchors)
top_idx = np.argsort(y_all)[-3:]
seed_points = X_all[top_idx]

# add small local perturbations
for seed in seed_points:
    for _ in range(5):
        x0 = np.clip(seed + 0.03 * RND.randn(3), 0, 1)
        res = minimize(
            lambda x: -expected_improvement(x, gp, y_best, xi=0.001),
            x0=x0,
            bounds=bounds,
            method="L-BFGS-B"
        )
        if res.success:
            ei_val = expected_improvement(res.x, gp, y_best, xi=0.001)
            if ei_val > best_ei:
                best_ei = ei_val
                best_x = res.x

# a few random seeds as safety
for _ in range(10):
    x0 = RND.rand(3)
    res = minimize(
        lambda x: -expected_improvement(x, gp, y_best, xi=0.001),
        x0=x0,
        bounds=bounds,
        method="L-BFGS-B"
    )
    if res.success:
        ei_val = expected_improvement(res.x, gp, y_best, xi=0.001)
        if ei_val > best_ei:
            best_ei = ei_val
            best_x = res.x

# -------------------------
# Report
# -------------------------
mu_next, sigma_next = gp.predict(best_x.reshape(1, -1), return_std=True)

print("\nNext point to evaluate (6 dp):", np.round(best_x, 6))
print("Predicted mu:", float(mu_next[0]))
print("Predicted sigma:", float(sigma_next[0]))
print("EI:", best_ei)


# Week 11
- The new point lies squarely inside the established “good basin”.
- Its performance is slightly worse than the current best, but much better than random exploration

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.stats import norm
from scipy.optimize import minimize

# --- Initial data ---
X_init = np.array([
    [0.17152521, 0.34391687, 0.2487372 ],
    [0.24211446, 0.64407427, 0.27243281],
    [0.53490572, 0.39850092, 0.17338873],
    [0.49258141, 0.61159319, 0.34017639],
    [0.13462167, 0.21991724, 0.45820622],
    [0.34552327, 0.94135983, 0.26936348],
    [0.15183663, 0.43999062, 0.99088187],
    [0.64550284, 0.39714294, 0.91977134],
    [0.74691195, 0.28419631, 0.22629985],
    [0.17047699, 0.6970324 , 0.14916943],
    [0.22054934, 0.29782524, 0.34355534],
    [0.66601366, 0.67198515, 0.2462953 ],
    [0.04680895, 0.23136024, 0.77061759],
    [0.60009728, 0.72513573, 0.06608864],
    [0.96599485, 0.86111969, 0.56682913]
])

y_init = np.array([
    -0.1121222 , -0.08796286, -0.11141465, -0.03483531, -0.04800758,
    -0.11062091, -0.39892551, -0.11386851, -0.13146061, -0.09418956,
    -0.04694741, -0.10596504, -0.11804826, -0.03637783, -0.05675837
])

# --- Add new observations from previous rounds ---
X_new = np.array([
    [0.901884, 0.273619, 0.723590],
    [0.393393, 0.209704, 0.929527],
    [0.117994, 0.128418, 0.478883],
    [0.442975, 0.132914, 0.276531],
    [0.683163, 0.340182, 0.318758],
    [0.514909, 0.429977, 0.441068],
    [0.376435, 0.333454, 0.472888],
    [0.597924, 0.348150, 0.493042],
    [0.484909, 0.399977, 0.441068],
    [0.599163, 0.483661, 0.474838]
])

y_new = np.array([
    -0.13917950276182944,
    -0.1691006942213739,
    -0.043036710546582216,
    -0.11152705861024295,
    -0.05490061720239002, 
    -0.0050467853497373665,
    -0.017389844227996235,
    -0.009890240573922564,
    -0.02634297118984294,
    -0.010299714267429757
])

# Combine all data
X_all = np.vstack([X_init, X_new])
y_all = np.concatenate([y_init, y_new])


By round 10, the strategy shifted toward exploitation, guided by a consistently high-performing region identified in earlier rounds. The GP kernel was tightened to reflect observed steep local behavior, and EI was biased toward predicted mean. The resulting evaluation confirmed the stability of the identified optimum basin, even though no further improvement was achieved.
- Changes: Lower xi further to 0.0005

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.stats import norm
from scipy.optimize import minimize

# -------------------------
# Reproducibility
# -------------------------
np.random.seed(0)
RND = np.random.RandomState(0)

# -------------------------
# Tightened kernel
# - shorter length_scale to model steep local variation
# - bounded to prevent collapse
# -------------------------
kernel = Matern(
    length_scale=0.15,
    length_scale_bounds=(0.05, 0.5),
    nu=2.5
)

gp = GaussianProcessRegressor(
    kernel=kernel,
    alpha=1e-4,                 # regularization to avoid sigma collapse
    normalize_y=True,
    n_restarts_optimizer=10,
    random_state=0
)

gp.fit(X_all, y_all)

print("Fitted kernel:", gp.kernel_)

# -------------------------
# Best observed value
# -------------------------
y_best = y_all.max()

# -------------------------
# Exploitation-leaning EI
# - very small xi
# -------------------------
def expected_improvement(x, gp, y_best, xi=0.001):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu, sigma = float(mu[0]), float(sigma[0])

    if sigma <= 1e-12:
        return 0.0

    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return ei

# -------------------------
# Optimization
# - seed heavily near best points
# -------------------------
bounds = [(0, 1), (0, 1), (0, 1)]

best_x, best_ei = None, -np.inf

# top 3 observed points (exploitation anchors)
top_idx = np.argsort(y_all)[-3:]
seed_points = X_all[top_idx]

# add small local perturbations
for seed in seed_points:
    for _ in range(5):
        x0 = np.clip(seed + 0.03 * RND.randn(3), 0, 1)
        res = minimize(
            lambda x: -expected_improvement(x, gp, y_best, xi=0.0005),
            x0=x0,
            bounds=bounds,
            method="L-BFGS-B"
        )
        if res.success:
            ei_val = expected_improvement(res.x, gp, y_best, xi=0.0005)
            if ei_val > best_ei:
                best_ei = ei_val
                best_x = res.x

# a few random seeds as safety
for _ in range(10):
    x0 = RND.rand(3)
    res = minimize(
        lambda x: -expected_improvement(x, gp, y_best, xi=0.001),
        x0=x0,
        bounds=bounds,
        method="L-BFGS-B"
    )
    if res.success:
        ei_val = expected_improvement(res.x, gp, y_best, xi=0.001)
        if ei_val > best_ei:
            best_ei = ei_val
            best_x = res.x

# -------------------------
# Report
# -------------------------
mu_next, sigma_next = gp.predict(best_x.reshape(1, -1), return_std=True)

print("\nNext point to evaluate (6 dp):", np.round(best_x, 6))
print("Predicted mu:", float(mu_next[0]))
print("Predicted sigma:", float(sigma_next[0]))
print("EI:", best_ei)
